In [2]:
from bs4 import BeautifulSoup
import numpy as np
import pandas as pd
from time import sleep
import requests
import json
import re

In [9]:
mydf = pd.read_csv('GFM_url_list.csv', sep = '\t')
mydf


,Unnamed: 0,Url,Category,Position
0,0,https://www.gofundme.com/f/justiceforjacobblake,Medical,0
1,1,https://www.gofundme.com/f/official-navajo-nat...,Medical,0
2,2,https://www.gofundme.com/f/help-a-front-line-n...,Medical,0
3,3,https://www.gofundme.com/f/Tommy-Rivers-Rest-Up,Medical,1
4,4,https://www.gofundme.com/f/brandon039s-medical...,Medical,1
...,...,...,...,...
881,881,https://www.gofundme.com/f/Juneteenth-women-of...,Wishes,22
882,882,https://www.gofundme.com/f/allterrain-wheelcha...,Wishes,22
883,883,https://www.gofundme.com/f/Marthasdaughter,Wishes,23
884,884,https://www.gofundme.com/f/x2s8dc-financial-di...,Wishes,23


In [5]:
headers = ["Url", "Category","Position", "Title", "Location","Amount_Raised", "Goal", "Number_of_Donators", "Length_of_Fundraising", "Shares", "Followers", "Text"]
mydf = mydf.reindex(columns = headers)

full_df = pd.DataFrame(columns = headers)
#need to scrape a single url now

In [11]:
def scrape_url(row_index):
    single_row = mydf.iloc[row_index]
    url = single_row["Url"]
    category = single_row["Category"]
    position = single_row["Position"]
    
    page = requests.get(url)
         
    soup = BeautifulSoup(page.text, 'lxml')

    try:
        content_dic = json.loads(re.sub(';$','',soup.find('script').text.split('= ')[1]))

        amount_raised = content_dic['feed']['campaign']['current_amount']
        
        goal = content_dic['feed']['campaign']['goal_amount']
        
        NumDonators = content_dic['feed']['campaign']['donation_count']
        
        timeFundraised = (pd.Timestamp.now() - (pd.to_datetime(content_dic['feed']['campaign']['created_at'])).astimezone(tz=None))
    except:
        amount_raised = np.nan
        goal = np.nan
        NumDonators = np.nan
        timeFundraised = np.nan
        
    
    title_container = soup.find_all("h1",{"class":"a-campaign-title"})#<h1 class="campaign-title">Help Rick Muchow Beat Cancer</h1>
    
    try:
        title = title_container[0].text
    except:
        title = np.nan
    
    text_container =  soup.find('meta', attrs={'name': 'description'})
    
    try:
        all_text = text_container['content']
    except:
        all_text = np.nan
    
    try:
        FB_shares_container = soup.find_all("strong", {"class":"js-share-count-text"})
        FB_shares = FB_shares_container[0].text.splitlines()
        FB_shares = FB_shares[1].replace(" ", "").replace("\xa0", "")
    except:
        FB_shares = np.nan
        
    try:
        hearts_container = soup.find_all("div", {"class":"campaign-sp campaign-sp--heart fave-num"})
        hearts = hearts_container[0].text
    except:
        hearts = np.nan
    
    try:
        content_dic = json.loads(re.sub(';$','',soup.find('script').text.split('= ')[1]))
        location = content_dic['feed']['campaign']['location']['city']
    except:
        location = np.nan
        
    temp_row = np.array([[url, category, position, title, location, amount_raised, goal, NumDonators, timeFundraised, FB_shares, hearts, all_text]])
    temp_df = pd.DataFrame(temp_row, columns = headers)
    
    return(temp_df)

In [13]:
def scrape_all_urls(file = 'v1.csv', start = 0, end = len(mydf)):
    for i in range(start, end):
        try:
            temp_df = scrape_url(i)
            temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)
            print("Scraping url %s" %(i+1))
        except:
            sleep(5)
            temp_df = scrape_url(i)
            temp_df.to_csv(file, mode = 'a',sep = '\t', header = False)
            print("Scraping url %s" %(i+1))

In [ ]:
scrape_all_urls()

Scraping url 1
Scraping url 2
Scraping url 3
Scraping url 4
Scraping url 5
Scraping url 6
Scraping url 7
Scraping url 8
Scraping url 9
Scraping url 10
Scraping url 11
Scraping url 12
Scraping url 13
Scraping url 14
Scraping url 15
Scraping url 16
Scraping url 17
Scraping url 18
Scraping url 19
Scraping url 20
Scraping url 21
Scraping url 22
Scraping url 23
Scraping url 24
Scraping url 25
Scraping url 26
Scraping url 27
Scraping url 28
Scraping url 29
Scraping url 30
Scraping url 31
Scraping url 32
Scraping url 33
Scraping url 34
Scraping url 35


In [10]:
mydf = pd.read_csv('v1.csv', sep = '\t')

In [11]:
mydf

,0,https://www.gofundme.com/f/justiceforjacobblake,Medical,0.1,Justice for Jacob Blake,"Kenosha, WI",2293990,3000000,72896,65 days 07:54:21.539398000,Unnamed: 10,Unnamed: 11,On August 23rd my son was shot multiple times in the back by a Kenosha Police Department officer… Julia Jackson needs your support for Justice for Jacob Blake
0,0,https://www.gofundme.com/f/official-navajo-nat...,Medical,0,Official Navajo Nation COVID-19 Relief Fund,"Window Rock, AZ",1830020,1000000,22018,176 days 16:47:32.075812000,NaN,NaN,\nThe Navajo Nation COVID-19 Fund has been est...
1,0,https://www.gofundme.com/f/help-a-front-line-n...,Medical,0,Help a front line nurse and baby get proper care,"Randolph, NJ",949069,1200000,18880,187 days 11:12:25.587315000,NaN,NaN,"On Sunday, April 12, Sylvia Leroy, a pregnant ..."
2,0,https://www.gofundme.com/f/Tommy-Rivers-Rest-Up,Medical,1,"Rest up, Tommy, we'll see you soon","Scottsdale, AZ",638275,1000000,10820,102 days 11:50:46.915886000,NaN,NaN,"First, thank you for being here. Tommy Rivers ..."
3,0,https://www.gofundme.com/f/brandon039s-medical...,Medical,1,OFFICIAL BRANDON SAENZ MEDICAL FUND,"Tyler, TX",570229,750000,24726,147 days 11:41:21.451980000,NaN,NaN,My name is Melissa Green and I am the mother o...
4,0,https://www.gofundme.com/f/helping-laura-and-k...,Medical,1,Helping Laura and Katherine get through my death.,"Eugene, OR",419553,200000,9663,26 days 11:45:13.809476000,NaN,NaN,"I am dying of lymphoma. My wife, Laura, has be..."
5,0,https://www.gofundme.com/f/help-mattress-mack-...,Medical,2,Help Mattress Mack Save Camp Hope,"Houston, TX",439284,1000000,1736,34 days 17:58:59.317454000,NaN,NaN,Mattress Mack invites you to help him SAVE CAM...
6,0,https://www.gofundme.com/f/rise-up-for-rhys,Medical,2,Rise Up for Rhys,"Redondo Beach, CA",395725,500000,3183,134 days 10:13:21.731712000,NaN,NaN,\n6 month old Rhys needs a life saving drug th...
7,0,https://www.gofundme.com/f/covid19-protest-rel...,Medical,2,COVID-19 Protest Relief Fund,"New York, NY",369620,400000,7659,148 days 11:57:31.101996000,NaN,NaN,Image by: Nam Y. Huh/AP\n\n\nAs thousands of p...
8,0,https://www.gofundme.com/f/mercan-a-can,Medical,3,Mercan'a can - Life for Mercan (SMA Tip 1-Type 1),"Chicago, IL",1384680,2400000,10075,85 days 17:40:37.478202000,NaN,NaN,Fundraiser campaign for my cousin’s beloved 7-...
9,0,https://www.gofundme.com/f/2succs-the-nolan-fa...,Medical,3,The Nolan Family,"Rancho Santa Margarita, CA",363998,500000,5565,47 days 20:44:15.029260000,NaN,NaN,Our beloved Nolan Family needs our help right ...
